# Intention parser:

In [ ]:
import json
from typing import List, Dict
from dataclasses import dataclass, field
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

@dataclass
class UserIntent:
    medical_conditions: List[str] = field(default_factory=list)
    dietary_restrictions: List[str] = field(default_factory=list)
    allergies: List[str] = field(default_factory=list)
    ingredients: List[str] = field(default_factory=list)
    cooking_style: List[str] = field(default_factory=list)

class OllamaIntentParser:
    def __init__(self, model_name: str = "llama3.2"):
        # Initialize the LangChain Ollama wrapper
        self.llm = OllamaLLM(
            model=model_name,
            temperature=0,  # Zero for consistent extraction
            format="json"   # Ensures the model speaks JSON
        )
        self.parser = JsonOutputParser()
        self.chain = self._build_chain()

    def _build_chain(self):
        system_instructions = """
You are a medical nutrition data extractor.
Your task is to analyze the user’s input and extract only explicitly stated information into the following categories:

- medical: Diagnosed medical or clinical conditions
(e.g., type_2_diabetes, hypertension)
- restrictions: Dietary patterns or restrictions
(e.g., keto, vegan, low_sodium)
- allergies: Food allergies or intolerances
(e.g., shellfish, gluten, lactose)
- ingredients: Foods or ingredients the user says they have or want to use
(e.g., chicken, kale, olive_oil)
- style: Cooking or preparation preferences
(e.g., under_30_min, slow_cooker, no_oven)

Rules & Constraints:

- Extract only what is explicitly mentioned by the user.
- Do NOT infer, assume, guess, or auto-complete missing information.
- Do NOT apply defaults or common assumptions.
- If a category is not explicitly mentioned, return an empty list [] for that category.
- Do not merge categories or reclassify information.
- Normalize all extracted values using snake_case.

Output must include all categories, even if their lists are empty.

Output Format:
Return a single structured object containing exactly these five keys, each mapped to a list.
        """
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", system_instructions),
            ("user", "Analyze this query: {query}")
        ])
        
        return prompt | self.llm | self.parser

    def parse(self, query: str) -> UserIntent:
        try:
            # Execute the LangChain chain
            result = self.chain.invoke({"query": query})
            
            return UserIntent(
                medical_conditions=result.get("medical", []),
                dietary_restrictions=result.get("restrictions", []),
                allergies=result.get("allergies", []),
                ingredients=result.get("ingredients", []),
                cooking_style=result.get("style", [])
            )
        except Exception as e:
            print(f"Error parsing with LangChain: {e}")
            return UserIntent()


if __name__ == "__main__":
    parser = OllamaIntentParser()
    
    user_input = "I have some chicken, salad and eggs and I'm looking for a quick breakfast with max 4 ingredients recipe. I have high blood pressure, diabetes and I can't eat tomatoes, onions."
    
    intent = parser.parse(user_input)
    
    print(f"Medical:      {intent.medical_conditions}")
    print(f"Restrictions: {intent.dietary_restrictions}")
    print(f"Allergies:    {intent.allergies}")
    print(f"Ingredients:  {intent.ingredients}")
    print(f"Style:        {intent.cooking_style}")

Medical:      ['high_blood_pressure', 'type_2_diabetes']
Restrictions: ['low_sodium']
Allergies:    ['tomatoes', 'onions']
Ingredients:  ['chicken', 'salad', 'eggs']
Style:        ['under_30_min']
